In [1]:
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor, Lambda
from torch.utils.data import DataLoader

/Users/tarineccleston/Documents/software-ds/generative-models-learning/gans-torch/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
import argparse
import os
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

In [5]:
# Set random seed for reproducibility
manualSeed = 999
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.use_deterministic_algorithms(True) # Needed for reproducible results

Random Seed:  999


In [13]:
mnist_fashion_df = np.loadtxt('../data/mnist_fashion/fashion-mnist_train.csv', skiprows=1, delimiter=',')

mnist_sneakers_df = mnist_fashion_df[mnist_fashion_df[:,0] == 7]
mnist_labels_df = mnist_sneakers_df[:, 0] 
mnist_images_df = mnist_sneakers_df[:, 1:]